# ComfyUI Colab

This notebook will install ComfyUI and its dependencies on Google Colab.

## Setup

First, let's set up the environment and install ComfyUI.

In [ ]:
#@title Setup ComfyUI
import subprocess
import sys
import os
from pathlib import Path

# Clone ComfyUI
if not os.path.exists("ComfyUI"):
    subprocess.run(["git", "clone", "https://github.com/comfyanonymous/ComfyUI.git"])

# Install requirements
subprocess.run([sys.executable, "-m", "pip", "install", "-r", "ComfyUI/requirements.txt"])

print("ComfyUI setup complete!")

In [ ]:
#@title Download Models
import subprocess
import os

# Create models directory
os.makedirs("ComfyUI/models/checkpoints", exist_ok=True)
os.makedirs("ComfyUI/models/vae", exist_ok=True)
os.makedirs("ComfyUI/models/loras", exist_ok=True)

# Download SD1.5 model
subprocess.run(["wget", "-c", "https://huggingface.co/Comfy-Org/stable-diffusion-v1-5-archive/resolve/main/v1-5-pruned-emaonly-fp16.safetensors", "-P", "ComfyUI/models/checkpoints/"])

print("Models downloaded!")

## Run ComfyUI

Choose one of the following methods to run ComfyUI:

In [ ]:
#@title Method 1: Run ComfyUI with Cloudflared (Recommended)
import subprocess
import threading
import time
import requests
from IPython.display import display, HTML

def start_comfyui():
    subprocess.run(["python", "main.py", "--listen", "0.0.0.0", "--port", "8188"], cwd="ComfyUI")

# Start ComfyUI in background
comfyui_thread = threading.Thread(target=start_comfyui)
comfyui_thread.daemon = True
comfyui_thread.start()

# Wait for ComfyUI to start
time.sleep(30)
print("ComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)")

# Install and run cloudflared
subprocess.run(["wget", "-q", "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64", "-O", "cloudflared"])
subprocess.run(["chmod", "+x", "cloudflared"])

# Start cloudflared tunnel
tunnel_process = subprocess.Popen(["./cloudflared", "tunnel", "--url", "http://localhost:8188"], 
                                   stdout=subprocess.PIPE, 
                                   stderr=subprocess.PIPE, 
                                   text=True)

# Wait for tunnel to establish
time.sleep(10)

# Get the tunnel URL
tunnel_output = tunnel_process.stderr.read()
if "trycloudflare.com" in tunnel_output:
    tunnel_url = tunnel_output.split("https://")[1].split("\n")[0]
    tunnel_url = "https://" + tunnel_url
    print(f"This is the URL to access ComfyUI: {tunnel_url}")
    display(HTML(f'<a href="{tunnel_url}" target="_blank">Open ComfyUI</a>'))
else:
    print("Cloudflared tunnel failed to establish. Try the localtunnel method below.")

In [ ]:
#@title Method 2: Run ComfyUI with Localtunnel
import subprocess
import threading
import time
import requests
from IPython.display import display, HTML

def start_comfyui():
    subprocess.run(["python", "main.py", "--listen", "0.0.0.0", "--port", "8188"], cwd="ComfyUI")

# Start ComfyUI in background
comfyui_thread = threading.Thread(target=start_comfyui)
comfyui_thread.daemon = True
comfyui_thread.start()

# Wait for ComfyUI to start
time.sleep(30)
print("ComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)")

# Install Node.js and localtunnel
subprocess.run(["curl", "-fsSL", "https://deb.nodesource.com/setup_18.x", "|", "bash", "-"])
subprocess.run(["apt-get", "install", "-y", "nodejs"])
subprocess.run(["npm", "install", "-g", "localtunnel"])

# Start localtunnel
tunnel_process = subprocess.Popen(["lt", "--port", "8188"], 
                                   stdout=subprocess.PIPE, 
                                   stderr=subprocess.PIPE, 
                                   text=True)

# Wait for tunnel to establish
time.sleep(10)

# Get the tunnel URL and password
tunnel_output = tunnel_process.stdout.read()
if "loca.lt" in tunnel_output:
    # Extract URL and password
    lines = tunnel_output.split("\n")
    url_line = [line for line in lines if "loca.lt" in line][0]
    password_line = [line for line in lines if "password" in line.lower() or "endpoint" in line.lower()][0]
    
    tunnel_url = url_line.split("your url is: ")[1].strip()
    password = password_line.split(": ")[1].strip()
    
    print(f"The password/endpoint ip for localtunnel is: {password}")
    print(f"your url is: {tunnel_url}")
    display(HTML(f'<a href="{tunnel_url}" target="_blank">Open ComfyUI</a>'))
    print(f"\nWhen the page opens, enter this password: {password}")
else:
    print("Localtunnel failed to establish. Try the iframe method below.")

In [ ]:
#@title Method 3: Run ComfyUI with Colab IFrame
import subprocess
import threading
import time
from IPython.display import display, HTML

def start_comfyui():
    subprocess.run(["python", "main.py", "--listen", "0.0.0.0", "--port", "8188"], cwd="ComfyUI")

# Start ComfyUI in background
comfyui_thread = threading.Thread(target=start_comfyui)
comfyui_thread.daemon = True
comfyui_thread.start()

# Wait for ComfyUI to start
time.sleep(30)
print("ComfyUI started!")

# Display ComfyUI in iframe
display(HTML('''
<iframe src="http://localhost:8188" width="100%" height="800"></iframe>
'''))